In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from collections import namedtuple

import codecs

In [2]:
# Start by connecting gdrive into the google colab

#from google.colab import drive

#drive.mount('/content/gdrive')

#1 load the data:
Load the data into the solution. Please mind the schema/data types of fields.

In [3]:
# reading the data from the google drive

df = pd.read_csv(
    "https://github.com/TanjaMenkovic/task/blob/main/data.zip?raw=true", 
    compression='zip',
    encoding = 'unicode_escape',
    sep=';'
    )
df.head()

,Lainaus ID,Pyörän ID,Lainausaika,Lainauspaikan ID,Lainauspaikka,Lainauksen koordinaatit,Lainauksen tyyppi,Palautusaika,Palautuspaikan ID,Palautuspaikka,...,Kokonaiskesto,Lainauksen hinta,Keskeytyksen hinta,Kokonaishinta,Kauppaehdot,Huoltolainaus,Arvostelu,Kulutettu energia Wh,Korjattu energia Wh,Matkan pituus m
0,592271,9,26.4.2021 7.20.12,40.0,JYNKKÄ,"62.855023, 27.642339",ebike,26.4.2021 7.40.39,23.0,VIESTIKATU,...,0:20:26,"1,05",NaN,"1,05",MIN-HINTA,0,NaN,"35,4","2,4",6190
1,592272,117,26.4.2021 7.29.21,20.0,TASAVALLANKATU,"62.881166, 27.663587",freebike,26.4.2021 7.44.48,12.0,MICROTEKNIA,...,0:15:27,"0,8",NaN,"0,8",MIN-HINTA,0,NaN,0,"1,7",2940
2,592275,46,26.4.2021 7.32.12,2.0,TORI,"62.8930869444, 27.6790419444",ebike,26.4.2021 7.32.56,2.0,TORI,...,0:00:44,"0,05",NaN,"0,05",MIN-HINTA,0,NaN,0,0,0
3,592276,20,26.4.2021 7.33.21,2.0,TORI,"62.8930869444, 27.6790419444",ebike,26.4.2021 7.40.17,23.0,VIESTIKATU,...,0:06:56,"0,35",NaN,"0,35",MIN-HINTA,0,NaN,"10,2","0,5",2090
4,592277,267,26.4.2021 7.35.50,2.0,TORI,"62.8930869444, 27.6790419444",ebike,26.4.2021 7.44.37,14.0,SNELLMANIA,...,0:08:47,"0,45",NaN,"0,45",MIN-HINTA,0,NaN,"18,2","1,1",2660


In [4]:
df.dtypes

Lainaus ID                    int64
Pyörän ID                     int64
Lainausaika                  object
Lainauspaikan ID            float64
Lainauspaikka                object
Lainauksen koordinaatit      object
Lainauksen tyyppi            object
Palautusaika                 object
Palautuspaikan ID           float64
Palautuspaikka               object
Palautuksen koordinaatit     object
Lainauksen kesto             object
Keskeytyksen kesto           object
Kokonaiskesto                object
Lainauksen hinta             object
Keskeytyksen hinta          float64
Kokonaishinta                object
Kauppaehdot                  object
Huoltolainaus                 int64
Arvostelu                   float64
Kulutettu energia Wh         object
Korjattu energia Wh          object
Matkan pituus m               int64
dtype: object

In [5]:
# changing date and time into datetime type:
df['Lainausaika'] = pd.to_datetime(df['Lainausaika'], format="%d.%m.%Y %H.%M.%S")
df['Palautusaika'] = pd.to_datetime(df['Palautusaika'], format="%d.%m.%Y %H.%M.%S")

In [ ]:
def dec(row, col):
  if pd.notna(row[col]):
    pom = row[col].replace(',', '.')
    pom = float(pom)
    return pom
  else:
    return None

df['Lainauksen hinta'] = df.apply(lambda row: dec(row, 'Lainauksen hinta'), axis=1)
df['Kokonaishinta'] = df.apply(lambda row: dec(row, 'Kokonaishinta'), axis=1)
df['Kulutettu energia Wh'] = df.apply(lambda row: dec(row, 'Kulutettu energia Wh'), axis=1)
df['Korjattu energia Wh'] = df.apply(lambda row: dec(row, 'Korjattu energia Wh'), axis=1)

In [ ]:
df.dtypes

#2 Warmup
How many trips/entries does the dataset contain? And how many unique bike ids?

In [ ]:
df.shape

In [ ]:
len(df)

In [ ]:
df['Pyörän ID'].unique().sum()

There is 344099 entries and 36585 unique bike ids.

#3 Trips
Fields

Bike id is Pyörän ID
Trip length in meters is Matkan pituus m
Questions

Which three bikes had the least trips? How many trips did each have?
Which three bikes had the most trips? How many trips did each have?
Which three bikes travelled the least distance during the year? Show the distance in kilometers.
Which bikes travelled over 4500 km during the season if any?

In [ ]:
df['Pyörän ID'].value_counts()

3 bikes with the least trips were:
* Bike with ID 226 has 4 trips.
* Bike with ID 173 has 578 trips.
* Bike with ID 254 has 605 trips.

3 bikes with the most trips were:
* Bike with ID 258 has 1659 trips.
* Bike with ID 142 has 1631 trips.
* Bike with ID 247 has 1593 trips.

In [ ]:
df['Matkan pituus km'] = df['Matkan pituus m']/1000
df.head()

In [ ]:
df_ID = df.groupby(by=['Pyörän ID']).sum()
df_ID['Matkan pituus km'].sort_values()

Three bikes travelled the least distance during the year:
* Bike ID 226 travelled 0km during the year
* Bike ID 254 travelled 1227.4km during the year
* Bike ID 173 travelled 1458.41km during the year

Bike that travelled over 4500 km during the season is with the ID 258 (4663.38km).

#4 Revenue
Trip revenue is Kokonaishinta
* Which bike made the most revenue?

In [ ]:
df_ID['Kokonaishinta'].sort_values()

Most revenue made bike with ID 95 (910.40)

#5 Maintenance
Field is Huoltolainaus which is 1 for maintenance.
* Count top 5 most maintained bikes. How many times were they taken to
maintenance?

In [ ]:
df_ID['Huoltolainaus'].sort_values()

Top 5 most maintained bikes are:
* Bike with ID 190 was taken to maintenance 131 times
* Bike with ID 172 was taken to maintenance 114 times
* Bike with ID 43 was taken to maintenance 113 times
* Bike with ID 181 was taken to maintenance 109 times
* Bike with ID 128 was taken to maintenance 104 times

#6 Distances
Start coordinates are in Lainauksen koordinaatit and return coords are in Palautuksen
koordinaatit.

Latitude and longitude for both loans and returns are stuffed in a single field in the dataset. Split them into separate fields and add those new fields into the dataset as new fields
preserving the originals. In addition, calculate the distance between start and end (in KM).
* Show 5 rows of the result
* Which bike was left farthest away from its starting point?
* What kind of discrepancies did you identify in the dataset while working with the
coordinates, if any?

In [ ]:
"""

def coordinate(row, col):
  if pd.notna(row[col]):
    lat, lon = row[col].split(', ')
    lat, lon = float(lat), float(lon)
    return lat, lon
  else:
    return None, None

df['Lainauksen lat'], df['Lainauksen lon'] = df.apply(lambda row: coordinate(row, 'Lainauksen koordinaatit'), axis=1)
df['Palautuksen koordinaatit'] = df['Palautuksen koordinaatit'].replace('#PUUTTUU!', None)
df['Palautuksen lat'], df['Palautuksen lon'] = df.apply(lambda row: coordinate(row, 'Palautuksen koordinaatit'), axis=1)
                                                        
df.head()
"""

In [ ]:
def latitude(row, col):
   if pd.notna(row[col]):
    lat, lon = row[col].split(', ')
    lat = float(lat)
    return lat
   else:
    return None

def longitude(row, col):
   if pd.notna(row[col]):
    lat, lon = row[col].split(', ')
    lon = float(lon)
    return lon
   else:
    return None

df['Lainauksen lat'] = df.apply(lambda row: latitude(row, 'Lainauksen koordinaatit'), axis=1)
df['Lainauksen lon'] = df.apply(lambda row: longitude(row, 'Lainauksen koordinaatit'), axis=1)

df['Palautuksen koordinaatit'] = df['Palautuksen koordinaatit'].replace('#PUUTTUU!', None)
df['Palautuksen lat'] = df.apply(lambda row: latitude(row, 'Palautuksen koordinaatit'), axis=1)
df['Palautuksen lon'] = df.apply(lambda row: longitude(row, 'Palautuksen koordinaatit'), axis=1)

df.head()

In [ ]:
from math import radians, cos, sin, asin, sqrt

def distance(row, lat1, lon1, lat2, lon2):
    if pd.notna(row[lat1]) & pd.notna(row[lat2]) & pd.notna(row[lon1]) & pd.notna(row[lon2]):
      ln1 = radians(row[lon1])
      ln2 = radians(row[lon2])
      lt1 = radians(row[lat1])
      lt2 = radians(row[lat2])
      
      # Haversine formula
      dlon = ln2 - ln1
      dlat = lt2 - lt1
      a = sin(dlat / 2)**2 + cos(lt1) * cos(lt2) * sin(dlon / 2)**2
 
      c = 2 * asin(sqrt(a))
    
      # Radius of earth in kilometers.
      r = 6371

      return(c * r)

    else:
      return None

df['Distance'] = df.apply(lambda row: distance(row, 'Lainauksen lat', 'Lainauksen lon', 'Palautuksen lat', 'Palautuksen lon'), axis=1)
df.head(5)

In [ ]:
df_new = df.dropna(subset=['Distance'])
df_new[['Pyörän ID', 'Distance']].sort_values(by='Distance')

Bike with ID 95 was left farthest away from its starting point (around 41km).

There are some missing values ​​in some rows and also some rows with value '#PUUTTUU!' in column 'Palautuksen koordinaatit', so I needed to treat it all as a None.

#7 Relational model
Design a potential relational model (entity relationship diagram) for a database in
which the database could be pushed. Tool choice is free.

In [ ]:
!pip install sqlalchemy

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
# Create the db engine
engine = create_engine('sqlite:///:memory:')

# Store the dataframe as a table
df.to_sql('data_table', engine)

# Query 1 on the relational table
res1 = pd.read_sql_query('SELECT * FROM data_table', engine)
print('Result 1')
print(res1)
print('')

# Query 2 on the relational table
res2 = pd.read_sql_query('SELECT dept,sum(salary) FROM data_table group by dept', engine)
print('Result 2')
print(res2)